In [21]:
import pandas as pd
import requests
pd.set_option('display.max_columns', None)
import time
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re
import datetime
import pickle
import json
import collections
import itertools

In [2]:
df = pd.read_excel("stats2022-2023REG.xlsx")
df

,Unnamed: 0,Player,Team,Age,GP,W,L,Min,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-
0,0,Jayson Tatum,BOS,25,74,52,22,2732.2,2225,727,1559,46.6,240,686,35.0,531,622,85.4,78,571,649,342,213,78,51,160,3691,31,1,470
1,1,Joel Embiid,PHI,29,66,43,23,2284.1,2183,728,1328,54.8,66,200,33.0,661,771,85.7,113,557,670,274,226,66,112,205,3706,39,1,424
2,2,Luka Doncic,DAL,24,66,33,33,2390.5,2138,719,1449,49.6,185,541,34.2,515,694,74.2,54,515,569,529,236,90,33,166,3747,36,10,128
3,3,Shai Gilgeous-Alexander,OKC,24,68,33,35,2416.0,2135,704,1381,51.0,58,168,34.5,669,739,90.5,59,270,329,371,192,112,65,192,3425,3,0,149
4,4,Giannis Antetokounmpo,MIL,28,63,47,16,2023.6,1959,707,1278,55.3,47,171,27.5,498,772,64.5,137,605,742,359,246,52,51,197,3451,46,6,341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,534,Alondes Williams,BKN,23,1,1,0,5.3,0,0,0,0.0,0,0,0.0,0,0,0.0,0,1,1,0,2,0,0,1,-1,0,0,-5
535,535,Deonte Burton,SAC,29,2,1,1,6.5,0,0,2,0.0,0,1,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,5
536,536,Frank Jackson,UTA,25,1,0,1,5.0,0,0,3,0.0,0,1,0.0,0,0,0.0,1,1,2,1,0,0,0,0,4,0,0,-2
537,537,Michael Foster Jr.,PHI,20,1,1,0,1.0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,-2


In [3]:
team_list = list(df.Team.unique())
team_list = ['BRK' if item == 'BKN' else item for item in team_list]
team_list = ['CHO' if item == 'CHA' else item for item in team_list]
team_list = ['PHO' if item == 'PHX' else item for item in team_list]
team_list

['BOS',
 'PHI',
 'DAL',
 'OKC',
 'MIL',
 'MIN',
 'NYK',
 'CLE',
 'ATL',
 'CHI',
 'POR',
 'SAC',
 'TOR',
 'UTA',
 'DEN',
 'HOU',
 'GSW',
 'BRK',
 'MEM',
 'LAL',
 'NOP',
 'MIA',
 'WAS',
 'ORL',
 'PHO',
 'SAS',
 'IND',
 'LAC',
 'CHO',
 'DET']

In [7]:
number = 1
driver = webdriver.Firefox()

power_rankings = {}

while(number <= 13):
    if(number == 2):
        url = 'https://www.nba.com/news/power-rankings-week-2-2'
    elif(number == 4 or number == 5):
        url = f"https://www.nba.com/news/power-rankings-week-{number}-la-clippers"
    elif(number == 6 or number == 8):
        url = f"https://www.nba.com/news/power-rankings-week-{number}-golden-state-warriors-cleveland-cavaliers"
    elif(number == 7):
        url = "https://www.nba.com/news/power-rankings-week-7-golden-state-warriors-san-antonio-spurs"
    elif(number == 9):
        url = "https://www.nba.com/news/power-rankings-week-9-houston-rockets-utah-jazz"
    elif(number == 10 or number == 11):
        url = f"https://www.nba.com/news/power-rankings-week-{number}-cleveland-cavaliers-golden-state-warriors"
    elif(number == 12):
        url = "https://www.nba.com/news/power-rankings-week-12-san-antonio-spurs-houston-rockets"
    else: 
        url = f"https://www.nba.com/news/power-rankings-week-{number}"
    driver.get(url)
    src = driver.page_source
    parser = BeautifulSoup(src, "lxml")
    ranks = parser.find_all("div", attrs = {"class": "ArticlePowerRankings_prTop__UrGiu"})
    published = parser.find('time', attrs = {"class": "ArticleHeader_ahDate__J3fwr"})
    time = published.contents[0]
    a_list = []
    for rank in ranks:
        a_tag = rank.find('a', href=True) 
        a_list.append(a_tag['href'])
    if('Updated on' in published.contents[0]):
        published.contents[0] = published.contents[0].replace('Updated on ', '')
    form = "%B %d, %Y %I:%M %p"
    new_date = datetime.datetime.strptime(published.contents[0], form).date()
    output_form = '%m-%d-%Y'
    processed_date = datetime.datetime.strftime(new_date, output_form)
    
    a_list = ['BRK' if item == '/nets' else item for item in a_list]
    a_list = ['CHO' if item == '/hornets/' else item for item in a_list]
    a_list = ['PHO' if item == '/suns/' else item for item in a_list]
    a_list = ['BOS' if item == '/celtics/' else item for item in a_list]
    a_list = ['MIL' if item == '/bucks/' else item for item in a_list]
    a_list = ['DEN' if item == '/nuggets/' else item for item in a_list]
    a_list = ['PHI' if item == '/sixers/' else item for item in a_list]
    a_list = ['CLE' if item == '/cavaliers' else item for item in a_list]
    a_list = ['MEM' if item == '/grizzlies/' else item for item in a_list]
    a_list = ['SAC' if item == '/kings' else item for item in a_list]
    a_list = ['NYK' if item == '/knicks/' else item for item in a_list]
    a_list = ['GSW' if item == '/warriors' else item for item in a_list]
    a_list = ['LAC' if item == '/clippers/' else item for item in a_list]
    a_list = ['LAL' if item == 'http://nba.com/lakers' else item for item in a_list]
    a_list = ['MIA' if item == '/heat/' else item for item in a_list]
    a_list = ['NOP' if item == '/pelicans' else item for item in a_list]
    a_list = ['OKC' if item == '/thunder' else item for item in a_list]
    a_list = ['MIN' if item == '/timberwolves' else item for item in a_list]
    a_list = ['CHI' if item == '/bulls' else item for item in a_list]
    a_list = ['TOR' if item == '/raptors/' else item for item in a_list]
    a_list = ['ATL' if item == '/hawks/' else item for item in a_list]
    a_list = ['DAL' if item == 'https://www.mavs.com' else item for item in a_list]
    a_list = ['UTA' if item == '/jazz/' else item for item in a_list]
    a_list = ['ORL' if item == '/magic' else item for item in a_list]
    a_list = ['WAS' if item == '/wizards' else item for item in a_list]
    a_list = ['IND' if item == '/pacers/' else item for item in a_list]
    a_list = ['POR' if item == '/blazers/' else item for item in a_list]
    a_list = ['HOU' if item == '/rockets' else item for item in a_list]
    a_list = ['SAS' if item == '/spurs/' else item for item in a_list]
    a_list = ['DET' if item == '/pistons' else item for item in a_list]
    
    power_rankings[processed_date] = a_list
    print('week ' + str(number) + ' done')
    
    number += 1
    
    
driver.close()
    

week 1 done
week 2 done
week 3 done
week 4 done
week 5 done
week 6 done
week 7 done
week 8 done
week 9 done
week 10 done
week 11 done
week 12 done


AttributeError: 'NoneType' object has no attribute 'contents'

In [150]:
with open("powerrankings17-18.json", "w") as f:
    json.dump(power_rankings, f)


In [40]:
date_dict = {}

In [41]:
driver = webdriver.Firefox()
start_date = datetime.date(2015, 10, 27)
end_date = datetime.date(2016, 4, 14)
delta = datetime.timedelta(days = 1)

#date_dict = {}

while(start_date < end_date):
    day2 = str(start_date.day)
    month2 = str(start_date.month)
    year2 = str(start_date.year)
    if(start_date.day < 10):
        day2 = "0" + day2
    if(start_date.month < 10):
        month2 = "0" + month2

    date = '' + month2 + '-' + day2 + '-' + year2  
    date_dict[date] = {}

    for team in team_list:
        url = f"https://www.basketball-reference.com/boxscores/{year2}{month2}{day2}0{team}.html"
        response = requests.get(url, headers = {'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',})
        time.sleep(3)
        print(url)
        if response.status_code == 429:
            print(int(response.headers["Retry-After"]))
            time.sleep(int(response.headers["Retry-After"]))
        try:
            response.raise_for_status()
        except requests.exceptions.HTTPError as e:
            continue

        driver.get(url)
        print("hi")
        time.sleep(8)
        src = driver.page_source
        soup = BeautifulSoup(src, 'lxml')

        tables = soup.find_all('table', {'id': re.compile('box-.*-game-basic')})
        advanced = soup.find_all('table', {'id': re.compile('box-.*-game-advanced')})

        away = tables[0].get('id')[4:7]

        dfs = pd.read_html(str(tables), header=1)
        advanced_dfs = pd.read_html(str(advanced), header = 1)

        df_away = dfs[0]
        df_home = dfs[1]
        df_away_advanced = advanced_dfs[0]
        df_home_advanced = advanced_dfs[1]

        df_away = pd.merge(df_away, df_away_advanced, on = ['Starters', 'MP'])
        df_home = pd.merge(df_home, df_home_advanced, on = ['Starters', 'MP'])
        
        df_away['Home'] = 0
        df_home['Home'] = 1

        date_dict[date][team] = {}
        date_dict[date][away] = {}

        date_dict[date][team][away] = df_home
        date_dict[date][away][team] = df_away

    print("day done")
    start_date += delta


driver.close()

https://www.basketball-reference.com/boxscores/201510270BOS.html
https://www.basketball-reference.com/boxscores/201510270PHI.html
https://www.basketball-reference.com/boxscores/201510270DAL.html
https://www.basketball-reference.com/boxscores/201510270OKC.html
https://www.basketball-reference.com/boxscores/201510270MIL.html
https://www.basketball-reference.com/boxscores/201510270MIN.html
https://www.basketball-reference.com/boxscores/201510270NYK.html
https://www.basketball-reference.com/boxscores/201510270CLE.html
https://www.basketball-reference.com/boxscores/201510270ATL.html
hi
https://www.basketball-reference.com/boxscores/201510270CHI.html
hi
https://www.basketball-reference.com/boxscores/201510270POR.html
https://www.basketball-reference.com/boxscores/201510270SAC.html
https://www.basketball-reference.com/boxscores/201510270TOR.html
https://www.basketball-reference.com/boxscores/201510270UTA.html
https://www.basketball-reference.com/boxscores/201510270DEN.html
https://www.basketb

https://www.basketball-reference.com/boxscores/201510310MIN.html
https://www.basketball-reference.com/boxscores/201510310NYK.html
https://www.basketball-reference.com/boxscores/201510310CLE.html
https://www.basketball-reference.com/boxscores/201510310ATL.html
https://www.basketball-reference.com/boxscores/201510310CHI.html
https://www.basketball-reference.com/boxscores/201510310POR.html
hi
https://www.basketball-reference.com/boxscores/201510310SAC.html
https://www.basketball-reference.com/boxscores/201510310TOR.html
https://www.basketball-reference.com/boxscores/201510310UTA.html
https://www.basketball-reference.com/boxscores/201510310DEN.html
https://www.basketball-reference.com/boxscores/201510310HOU.html
https://www.basketball-reference.com/boxscores/201510310GSW.html
https://www.basketball-reference.com/boxscores/201510310BRK.html
https://www.basketball-reference.com/boxscores/201510310MEM.html
hi
https://www.basketball-reference.com/boxscores/201510310LAL.html
https://www.basketb

https://www.basketball-reference.com/boxscores/201511040POR.html
https://www.basketball-reference.com/boxscores/201511040SAC.html
https://www.basketball-reference.com/boxscores/201511040TOR.html
https://www.basketball-reference.com/boxscores/201511040UTA.html
hi
https://www.basketball-reference.com/boxscores/201511040DEN.html
https://www.basketball-reference.com/boxscores/201511040HOU.html
hi
https://www.basketball-reference.com/boxscores/201511040GSW.html
hi
https://www.basketball-reference.com/boxscores/201511040BRK.html
https://www.basketball-reference.com/boxscores/201511040MEM.html
https://www.basketball-reference.com/boxscores/201511040LAL.html
https://www.basketball-reference.com/boxscores/201511040NOP.html
https://www.basketball-reference.com/boxscores/201511040MIA.html
https://www.basketball-reference.com/boxscores/201511040WAS.html
hi
https://www.basketball-reference.com/boxscores/201511040ORL.html
https://www.basketball-reference.com/boxscores/201511040PHO.html
hi
https://ww

https://www.basketball-reference.com/boxscores/201511080DEN.html
https://www.basketball-reference.com/boxscores/201511080HOU.html
https://www.basketball-reference.com/boxscores/201511080GSW.html
https://www.basketball-reference.com/boxscores/201511080BRK.html
https://www.basketball-reference.com/boxscores/201511080MEM.html
https://www.basketball-reference.com/boxscores/201511080LAL.html
https://www.basketball-reference.com/boxscores/201511080NOP.html
https://www.basketball-reference.com/boxscores/201511080MIA.html
hi
https://www.basketball-reference.com/boxscores/201511080WAS.html
https://www.basketball-reference.com/boxscores/201511080ORL.html
https://www.basketball-reference.com/boxscores/201511080PHO.html
https://www.basketball-reference.com/boxscores/201511080SAS.html
https://www.basketball-reference.com/boxscores/201511080IND.html
https://www.basketball-reference.com/boxscores/201511080LAC.html
https://www.basketball-reference.com/boxscores/201511080CHO.html
https://www.basketball

https://www.basketball-reference.com/boxscores/201511120LAL.html
https://www.basketball-reference.com/boxscores/201511120NOP.html
https://www.basketball-reference.com/boxscores/201511120MIA.html
hi
https://www.basketball-reference.com/boxscores/201511120WAS.html
https://www.basketball-reference.com/boxscores/201511120ORL.html
https://www.basketball-reference.com/boxscores/201511120PHO.html
hi
https://www.basketball-reference.com/boxscores/201511120SAS.html
https://www.basketball-reference.com/boxscores/201511120IND.html
https://www.basketball-reference.com/boxscores/201511120LAC.html
https://www.basketball-reference.com/boxscores/201511120CHO.html
https://www.basketball-reference.com/boxscores/201511120DET.html
day done
https://www.basketball-reference.com/boxscores/201511130BOS.html
hi
https://www.basketball-reference.com/boxscores/201511130PHI.html
https://www.basketball-reference.com/boxscores/201511130DAL.html
hi
https://www.basketball-reference.com/boxscores/201511130OKC.html
hi
h

...

https://www.basketball-reference.com/boxscores/201603240ORL.html
https://www.basketball-reference.com/boxscores/201603240PHO.html
https://www.basketball-reference.com/boxscores/201603240SAS.html
https://www.basketball-reference.com/boxscores/201603240IND.html
hi
https://www.basketball-reference.com/boxscores/201603240LAC.html
hi
https://www.basketball-reference.com/boxscores/201603240CHO.html
https://www.basketball-reference.com/boxscores/201603240DET.html
day done
https://www.basketball-reference.com/boxscores/201603250BOS.html
https://www.basketball-reference.com/boxscores/201603250PHI.html
https://www.basketball-reference.com/boxscores/201603250DAL.html
https://www.basketball-reference.com/boxscores/201603250OKC.html
https://www.basketball-reference.com/boxscores/201603250MIL.html
https://www.basketball-reference.com/boxscores/201603250MIN.html
https://www.basketball-reference.com/boxscores/201603250NYK.html
https://www.basketball-reference.com/boxscores/201603250CLE.html
https://ww

https://www.basketball-reference.com/boxscores/201603280LAC.html
hi
https://www.basketball-reference.com/boxscores/201603280CHO.html
https://www.basketball-reference.com/boxscores/201603280DET.html
day done
https://www.basketball-reference.com/boxscores/201603290BOS.html
https://www.basketball-reference.com/boxscores/201603290PHI.html
hi
https://www.basketball-reference.com/boxscores/201603290DAL.html
https://www.basketball-reference.com/boxscores/201603290OKC.html
https://www.basketball-reference.com/boxscores/201603290MIL.html
https://www.basketball-reference.com/boxscores/201603290MIN.html
https://www.basketball-reference.com/boxscores/201603290NYK.html
https://www.basketball-reference.com/boxscores/201603290CLE.html
hi
https://www.basketball-reference.com/boxscores/201603290ATL.html
https://www.basketball-reference.com/boxscores/201603290CHI.html
https://www.basketball-reference.com/boxscores/201603290POR.html
https://www.basketball-reference.com/boxscores/201603290SAC.html
https:/

https://www.basketball-reference.com/boxscores/201604020PHI.html
hi
https://www.basketball-reference.com/boxscores/201604020DAL.html
https://www.basketball-reference.com/boxscores/201604020OKC.html
https://www.basketball-reference.com/boxscores/201604020MIL.html
https://www.basketball-reference.com/boxscores/201604020MIN.html
https://www.basketball-reference.com/boxscores/201604020NYK.html
https://www.basketball-reference.com/boxscores/201604020CLE.html
https://www.basketball-reference.com/boxscores/201604020ATL.html
https://www.basketball-reference.com/boxscores/201604020CHI.html
hi
https://www.basketball-reference.com/boxscores/201604020POR.html
hi
https://www.basketball-reference.com/boxscores/201604020SAC.html
https://www.basketball-reference.com/boxscores/201604020TOR.html
https://www.basketball-reference.com/boxscores/201604020UTA.html
https://www.basketball-reference.com/boxscores/201604020DEN.html
hi
https://www.basketball-reference.com/boxscores/201604020HOU.html
https://www.b

https://www.basketball-reference.com/boxscores/201604060NYK.html
hi
https://www.basketball-reference.com/boxscores/201604060CLE.html
https://www.basketball-reference.com/boxscores/201604060ATL.html
https://www.basketball-reference.com/boxscores/201604060CHI.html
https://www.basketball-reference.com/boxscores/201604060POR.html
hi
https://www.basketball-reference.com/boxscores/201604060SAC.html
https://www.basketball-reference.com/boxscores/201604060TOR.html
https://www.basketball-reference.com/boxscores/201604060UTA.html
https://www.basketball-reference.com/boxscores/201604060DEN.html
https://www.basketball-reference.com/boxscores/201604060HOU.html
https://www.basketball-reference.com/boxscores/201604060GSW.html
https://www.basketball-reference.com/boxscores/201604060BRK.html
https://www.basketball-reference.com/boxscores/201604060MEM.html
https://www.basketball-reference.com/boxscores/201604060LAL.html
hi
https://www.basketball-reference.com/boxscores/201604060NOP.html
https://www.bask

https://www.basketball-reference.com/boxscores/201604100SAC.html
https://www.basketball-reference.com/boxscores/201604100TOR.html
https://www.basketball-reference.com/boxscores/201604100UTA.html
https://www.basketball-reference.com/boxscores/201604100DEN.html
hi
https://www.basketball-reference.com/boxscores/201604100HOU.html
hi
https://www.basketball-reference.com/boxscores/201604100GSW.html
https://www.basketball-reference.com/boxscores/201604100BRK.html
https://www.basketball-reference.com/boxscores/201604100MEM.html
https://www.basketball-reference.com/boxscores/201604100LAL.html
https://www.basketball-reference.com/boxscores/201604100NOP.html
https://www.basketball-reference.com/boxscores/201604100MIA.html
hi
https://www.basketball-reference.com/boxscores/201604100WAS.html
hi
https://www.basketball-reference.com/boxscores/201604100ORL.html
https://www.basketball-reference.com/boxscores/201604100PHO.html
https://www.basketball-reference.com/boxscores/201604100SAS.html
hi
https://ww

In [27]:
date_dict.keys()

354

In [42]:
with open('dict15-16.pkl', 'wb') as f:  # open a text file
    pickle.dump(date_dict, f)
    f.close()

In [43]:
with open('boxREG.pkl', 'wb') as f:  # open a text file
    pickle.dump(teams_dict, f)
    f.close()

In [9]:
#seasons = ['2012-13', '2013-14', '2014-15', '2015-16','2016-17','2017-18' ,'2018-19' ,'2019-20','2021-22', '2022-23']

In [23]:
#for season in seasons:
    #print(f"my name is {season}")

my name is 2012-2013
my name is 2013-2014
my name is 2014-2015
my name is 2015-2016
my name is 2016-2017
my name is 2017-2018
my name is 2018-2019
my name is 2019-2020
my name is 2021-2022
my name is 2022-2023


In [10]:
#timeout = 25

In [11]:
#driver = webdriver.Firefox()

In [15]:
'''def get_data(seasons):
    for season in seasons:
        url = f"https://www.nba.com/stats/teams/misc?Season={season}&SeasonType=Regular+Season"
        driver.get(url)
        #element_present = EC.presence_of_element_located((By.XPATH, r"/html/body/div[1]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select"))
        #WebDriverWait(driver, timeout).until(element_present)
        #select = Select(driver.find_element("xpath", r"/html/body/div[1]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select"))
        #select.select_by_index(0)
        src = driver.page_source
        parser = BeautifulSoup(src, "lxml")
        table = parser.find("table", attrs = {"class": "Crom_table__p1iZz"})
        headers = table.findAll('th')
        headerlist = [h.text.strip() for h in headers[1:]]
        filter_header_list = [a for a in headerlist if not 'RANK' in a]
        rows = table.findAll('tr')[1:]
        player_stats = [[td.getText().strip() for td in rows[i].findAll('td')[1:]] for i in range(len(rows))]
        #filter_header_list = filter_header_list[:-5]
        stats = pd.DataFrame(player_stats, columns = filter_header_list)
        #pd.DataFrame.to_excel(stats, f"/home/rohan/python-projects/jupyter/BasketballProject/miscteam{season}REG.xlsx")
        print(season + " done scraping")
''' 

In [16]:
#get_data(seasons)

2012-13 done scraping
2013-14 done scraping
2014-15 done scraping


AttributeError: 'NoneType' object has no attribute 'findAll'